In [ ]:
"""
ADVANCED Supply Chain Resilience & Disruption Modeling System
Enterprise-Grade Implementation with Advanced Analytics

NEW FEATURES:
- Multi-objective optimization (cost vs service level)
- Time-series forecasting with ARIMA/Prophet
- Network graph analysis for supply chain topology
- Monte Carlo simulation for risk quantification
- Scenario analysis (what-if modeling)
- A/B testing framework for strategy validation
- Real-time anomaly detection
- Supplier diversification recommendations
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# ML & Advanced Analytics
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.ensemble import IsolationForest
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
from matplotlib.backends.backend_pdf import PdfPages


# Network Analysis
import networkx as nx

# Time Series (if available - will gracefully handle if not installed)
try:
    from statsmodels.tsa.arima.model import ARIMA
    from statsmodels.tsa.seasonal import seasonal_decompose
    TIMESERIES_AVAILABLE = True
except ImportError:
    TIMESERIES_AVAILABLE = False


# ============================================================================
# PHASE 1: DATA INGESTION, MAPPING & NETWORK TOPOLOGY
# ============================================================================

class AdvancedSupplyChainDataEngineer:
    """Handles Kaggle data loading, mapping, and network analysis"""
    
    def __init__(self):
        self.data = None
        self.network = None
        
    def load_and_map_data(self, file_path):
        """Loads the CSV from Kaggle path and maps columns to system standards"""
        print(f"\n📂 Loading dataset from Kaggle: {file_path}")
        
        # Load the raw data
        df = pd.read_csv(file_path)
        
        # 1. Map your specific CSV columns to the system's required names
        # We map Buyer_ID to Country and Organization_ID to Warehouse to maintain 
        # the geographical/structural logic of the network.
        mapping = {
            'Buyer_ID': 'Destination_Country',
            'Organization_ID': 'Origin_Warehouse', 
            'Quantity_Ordered': 'Order_Quantity',
            'Order_Value_USD': 'Sales_Value',
            'Supply_Risk_Flag': 'Delivery_Status'
        }
        df = df.rename(columns=mapping)
        
        # 2. Temporal Data Processing
        df['Order_Date'] = pd.to_datetime(df['Order_Date'])
        df['Delivery_Date'] = pd.to_datetime(df['Delivery_Date'])
        
        # Calculate real lead time in days
        df['Days_Real'] = (df['Delivery_Date'] - df['Order_Date']).dt.days.clip(lower=1)
        
        # Calculate scheduled days by subtracting your 'Delay_Days' from 'Days_Real'
        df['Days_Scheduled'] = (df['Days_Real'] - df['Delay_Days']).clip(lower=1)
        
        # 3. Fill missing fields required for subsequent modeling
        if 'Transport_Cost' not in df.columns:
            # Estimate freight as 8% of value adjusted by Shipping Mode
            mode_multiplier = {'Rail': 1.1, 'Road': 1.0, 'Air': 2.5, 'Sea': 0.7}
            df['Transport_Cost'] = df['Sales_Value'] * 0.08 * df['Shipping_Mode'].map(mode_multiplier).fillna(1.0)
            
        if 'Benefit_per_Order' not in df.columns:
            df['Benefit_per_Order'] = df['Sales_Value'] * 0.25 # Assuming a 25% margin
            
        if 'Unit_Price' not in df.columns:
            df['Unit_Price'] = df['Sales_Value'] / (df['Order_Quantity'] + 1)
            
        if 'Customer_Priority' not in df.columns:
            # Leverage your 'Dominant_Buyer_Flag' to assign priority
            df['Customer_Priority'] = df['Dominant_Buyer_Flag'].map({1: 'Enterprise', 0: 'Standard'})
            
        self.data = df
        print(f"✓ Successfully mapped {len(df):,} records from Kaggle source.")
        return self.data

    def build_supply_chain_network(self):
        """Builds a directed graph: Supplier -> Organization -> Buyer"""
        print("\n" + "=" * 80)
        print("SUPPLY CHAIN NETWORK TOPOLOGY ANALYSIS")
        print("=" * 80)
        
        G = nx.DiGraph()
        
        # Edges from Supplier to the Internal Organization (Warehouse)
        sw = self.data.groupby(['Supplier_ID', 'Origin_Warehouse']).size().reset_index(name='weight')
        for _, row in sw.iterrows():
            G.add_edge(row['Supplier_ID'], row['Origin_Warehouse'], weight=row['weight'])
            
        # Edges from the Organization to the Final Buyer (Destination)
        wc = self.data.groupby(['Origin_Warehouse', 'Destination_Country']).size().reset_index(name='weight')
        for _, row in wc.iterrows():
            G.add_edge(row['Origin_Warehouse'], row['Destination_Country'], weight=row['weight'])
            
        self.network = G
        print(f"📊 Network Map Created: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
        return G

    def engineer_advanced_features(self):
        """Generates the 30+ features required for the predictive models"""
        df = self.data.copy()
        
        # Temporal Features
        df['Month'] = df['Order_Date'].dt.month
        df['Quarter'] = df['Order_Date'].dt.quarter
        df['Day_of_Week'] = df['Order_Date'].dt.dayofweek
        df['Is_Peak_Season'] = df['Quarter'].isin([4]).astype(int)
        df['Is_Winter'] = df['Month'].isin([12, 1, 2]).astype(int)
        
        # Volatility & Historical Risk Scores
        df['Route_Volatility'] = df.groupby(['Destination_Country', 'Shipping_Mode'])['Days_Real'].transform('std').fillna(0)
        df['Country_Risk_Score'] = df.groupby('Destination_Country')['Delivery_Status'].transform('mean')
        df['Mode_Risk_Score'] = df.groupby('Shipping_Mode')['Delivery_Status'].transform('mean')
        df['Supplier_Risk_Score'] = df.groupby('Supplier_ID')['Delivery_Status'].transform('mean')
        
        # Sort for rolling window calculations
        df = df.sort_values('Order_Date')
        df['MA_30_Delay_Rate'] = df.groupby('Destination_Country')['Delivery_Status'].transform(lambda x: x.rolling(30, 1).mean())
        df['MA_90_Delay_Rate'] = df.groupby('Destination_Country')['Delivery_Status'].transform(lambda x: x.rolling(90, 1).mean())
        df['EMA_Delay'] = df.groupby('Destination_Country')['Days_Real'].transform(lambda x: x.ewm(span=20).mean())
        
        # Interaction Ratios
        df['Value_Quantity_Ratio'] = df['Sales_Value'] / (df['Order_Quantity'] + 1)
        df['Cost_Benefit_Ratio'] = df['Transport_Cost'] / (df['Benefit_per_Order'] + 1)
        df['Priority_Risk'] = df['Customer_Priority'].map({'Standard': 1, 'Premium': 2, 'Enterprise': 3}).fillna(1)
        
        # Centrality metrics from the network
        if self.network:
            bc = nx.betweenness_centrality(self.network)
            df['Supplier_Centrality'] = df['Supplier_ID'].map(bc).fillna(0)
            df['Warehouse_Centrality'] = df['Origin_Warehouse'].map(bc).fillna(0)
            
        self.data = df
        return df

    def detect_anomalies(self):
        """Identifies outlier shipments using Isolation Forest"""
        from sklearn.ensemble import IsolationForest
        # We focus on real lead time and value for anomaly detection
        features = ['Days_Real', 'Sales_Value', 'Transport_Cost']
        iso = IsolationForest(contamination=0.05, random_state=42)
        self.data['Is_Anomaly'] = iso.fit_predict(self.data[features].fillna(0))
        self.data['Is_Anomaly'] = (self.data['Is_Anomaly'] == -1).astype(int)
        return self.data[self.data['Is_Anomaly'] == 1]


# ============================================================================
# PHASE 2: ADVANCED PREDICTIVE MODELING & TIME SERIES
# ============================================================================

class AdvancedPredictiveModeling:
    """Enhanced ML with ensemble methods, hyperparameter tuning, and forecasting"""
    
    def __init__(self, data):
        self.data = data
        self.model = None
        self.best_params = None
        self.feature_importance = None
        self.forecast_results = None
        
    def prepare_modeling_data(self):
        """Prepare comprehensive feature set"""
        df = self.data.copy()
        
        # Feature selection
        feature_cols = ['Destination_Country', 'Shipping_Mode', 'Product_Category', 'Supplier_ID',
                       'Days_Scheduled', 'Order_Quantity', 'Sales_Value', 'Transport_Cost',
                       'Route_Volatility', 'Country_Risk_Score', 'Mode_Risk_Score', 
                       'Supplier_Risk_Score', 'Month', 'Quarter', 'Day_of_Week',
                       'Is_Peak_Season', 'Is_Winter', 'MA_30_Delay_Rate', 'MA_90_Delay_Rate',
                       'Value_Quantity_Ratio', 'Cost_Benefit_Ratio', 'Priority_Risk', 'EMA_Delay']
        
        if 'Supplier_Centrality' in df.columns:
            feature_cols.extend(['Supplier_Centrality', 'Warehouse_Centrality'])
        
        X = df[feature_cols].copy()
        y = df['Delivery_Status']
        
        # Encode categoricals
        label_encoders = {}
        for col in ['Destination_Country', 'Shipping_Mode', 'Product_Category', 'Supplier_ID']:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col].astype(str))
            label_encoders[col] = le
        
        # Handle any remaining nulls
        X = X.fillna(X.median())
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        print("\n" + "=" * 80)
        print("ADVANCED MODELING DATA PREPARATION")
        print("=" * 80)
        print(f"Training set: {len(self.X_train):,} | Test set: {len(self.X_test):,}")
        print(f"Features: {len(feature_cols)} advanced features")
        print(f"Class imbalance: {y.mean()*100:.2f}% delays")
        
        return self.X_train, self.X_test, self.y_train, self.y_test
    
    def train_with_hyperparameter_tuning(self):
        """GridSearchCV for optimal hyperparameters"""
        print("\n" + "=" * 80)
        print("HYPERPARAMETER OPTIMIZATION (GridSearchCV)")
        print("=" * 80)
        
        param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [4, 6, 8],
            'learning_rate': [0.05, 0.1],
            'subsample': [0.8, 0.9],
            'colsample_bytree': [0.8, 0.9]
        }
        
        xgb = XGBClassifier(random_state=42, eval_metric='logloss')
        
        print("\n⚙️  Running grid search with 3-fold cross-validation...")
        print(f"   Testing {np.prod([len(v) for v in param_grid.values()])} combinations")
        
        grid_search = GridSearchCV(
            xgb, param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=0
        )
        
        grid_search.fit(self.X_train, self.y_train)
        
        self.model = grid_search.best_estimator_
        self.best_params = grid_search.best_params_
        
        print(f"\n✓ Optimization complete!")
        print(f"  Best CV ROC-AUC: {grid_search.best_score_:.4f}")
        print(f"  Best parameters: {self.best_params}")
        
        return self.model
    
    def advanced_model_evaluation(self):
        """Comprehensive evaluation with multiple metrics"""
        print("\n" + "=" * 80)
        print("ADVANCED MODEL PERFORMANCE ANALYSIS")
        print("=" * 80)
        
        y_pred = self.model.predict(self.X_test)
        y_pred_proba = self.model.predict_proba(self.X_test)[:, 1]
        
        # Classification metrics
        print("\nClassification Report:")
        print(classification_report(self.y_test, y_pred, target_names=['On-Time', 'Late']))
        
        # ROC-AUC
        roc_auc = roc_auc_score(self.y_test, y_pred_proba)
        print(f"\nROC-AUC Score: {roc_auc:.4f}")
        
        # Feature importance
        self.feature_importance = pd.DataFrame({
            'Feature': self.X_train.columns,
            'Importance': self.model.feature_importances_
        }).sort_values('Importance', ascending=False)
        
        print("\n" + "-" * 80)
        print("TOP 15 PREDICTIVE FEATURES")
        print("-" * 80)
        print(self.feature_importance.head(15).to_string(index=False))
        
        # Prediction confidence analysis
        high_confidence = np.sum((y_pred_proba < 0.3) | (y_pred_proba > 0.7))
        print(f"\n📊 Prediction Confidence:")
        print(f"   High confidence predictions: {high_confidence}/{len(y_pred_proba)} ({high_confidence/len(y_pred_proba)*100:.1f}%)")
        
        return y_pred, y_pred_proba, self.feature_importance
    
    def time_series_forecasting(self):
        """Forecast future delay rates using time series analysis"""
        if not TIMESERIES_AVAILABLE:
            print("\n⚠️  Time series libraries not available. Skipping forecasting.")
            return None
        
        print("\n" + "=" * 80)
        print("TIME SERIES FORECASTING (30-Day Ahead)")
        print("=" * 80)
        
        # Aggregate daily delay rates
        daily_delays = self.data.groupby('Order_Date')['Delivery_Status'].mean()
        daily_delays = daily_delays.asfreq('D').fillna(method='ffill')
        
        # Train ARIMA model
        try:
            model = ARIMA(daily_delays, order=(5, 1, 2))
            fitted_model = model.fit()
            
            # Forecast next 30 days
            forecast = fitted_model.forecast(steps=30)
            
            print(f"\n✓ ARIMA(5,1,2) model fitted")
            print(f"  Current delay rate: {daily_delays.iloc[-1]*100:.2f}%")
            print(f"  30-day forecast (avg): {forecast.mean()*100:.2f}%")
            print(f"  Trend: {'📈 Increasing' if forecast.mean() > daily_delays.iloc[-1] else '📉 Decreasing'}")
            
            self.forecast_results = {
                'historical': daily_delays,
                'forecast': forecast
            }
            
            return forecast
        except Exception as e:
            print(f"⚠️  Forecasting error: {e}")
            return None


# ============================================================================
# PHASE 3: MULTI-OBJECTIVE OPTIMIZATION & MONTE CARLO SIMULATION
# ============================================================================

class AdvancedInventoryOptimization:
    """Sophisticated optimization with multiple objectives and risk quantification"""
    
    def __init__(self, data, model):
        self.data = data
        self.model = model
        self.simulation_results = None
        
    def multi_objective_optimization(self):
        """Optimize for both cost AND service level"""
        print("\n" + "=" * 80)
        print("MULTI-OBJECTIVE OPTIMIZATION")
        print("=" * 80)
        
        # Route-level aggregation
        routes = self.data.groupby(['Destination_Country', 'Shipping_Mode']).agg({
            'Order_Quantity': ['mean', 'std'],
            'Days_Real': ['mean', 'std'],
            'Delivery_Status': 'mean',
            'Sales_Value': 'sum',
            'Order_ID': 'count'
        }).reset_index()
        
        routes.columns = ['Country', 'Mode', 'Avg_Demand', 'Std_Demand', 
                         'Avg_LT', 'Std_LT', 'Delay_Rate', 'Total_Revenue', 'Volume']
        
        # Define objective function
        def objective(service_levels, holding_cost_per_unit=0.50, stockout_penalty=0.25):
            total_cost = 0
            total_service = 0
            
            for i, sl in enumerate(service_levels):
                z_score = stats.norm.ppf(sl)
                route = routes.iloc[i]
                
                # Safety stock calculation
                ss = z_score * np.sqrt(
                    route['Avg_LT'] * (route['Std_Demand'] ** 2) + 
                    (route['Avg_Demand'] ** 2) * (route['Std_LT'] ** 2)
                )
                
                # Holding cost
                holding = ss * holding_cost_per_unit * 30
                
                # Expected stockout cost
                stockout_prob = 1 - sl
                stockout_cost = stockout_prob * route['Avg_Demand'] * route['Total_Revenue']/route['Volume'] * stockout_penalty
                
                total_cost += holding + stockout_cost
                total_service += sl * route['Volume']
            
            # Multi-objective: minimize cost while maximizing service
            # Weight: 70% cost, 30% service level
            return 0.7 * total_cost - 0.3 * total_service
        
        # Constraints: service level between 0.85 and 0.99
        bounds = [(0.85, 0.99) for _ in range(len(routes))]
        initial_guess = [0.95] * len(routes)
        
        print("\n🔍 Running constrained optimization...")
        result = minimize(objective, initial_guess, method='L-BFGS-B', bounds=bounds)
        
        routes['Optimal_Service_Level'] = result.x
        routes['Service_Level_Category'] = pd.cut(routes['Optimal_Service_Level'], 
                                                   bins=[0.84, 0.90, 0.95, 1.0],
                                                   labels=['Standard', 'Enhanced', 'Premium'])
        
        print(f"\n✓ Optimization converged: {result.success}")
        print(f"  Objective value: {result.fun:,.2f}")
        
        print("\n" + "-" * 80)
        print("OPTIMIZED SERVICE LEVELS (Top 10 Routes by Revenue)")
        print("-" * 80)
        display = routes.nlargest(10, 'Total_Revenue')[['Country', 'Mode', 'Delay_Rate', 
                                                         'Optimal_Service_Level', 'Service_Level_Category']]
        print(display.to_string(index=False))
        
        return routes
    
    def monte_carlo_risk_simulation(self, n_simulations=10000):
        """Monte Carlo simulation for risk quantification"""
        print("\n" + "=" * 80)
        print(f"MONTE CARLO SIMULATION ({n_simulations:,} iterations)")
        print("=" * 80)
        
        # Select high-risk route for simulation
        high_risk_route = self.data[self.data['Delivery_Status'] == 1].groupby(
            ['Destination_Country', 'Shipping_Mode']
        ).size().idxmax()
        
        route_data = self.data[
            (self.data['Destination_Country'] == high_risk_route[0]) & 
            (self.data['Shipping_Mode'] == high_risk_route[1])
        ]
        
        avg_demand = route_data['Order_Quantity'].mean()
        std_demand = route_data['Order_Quantity'].std()
        avg_lt = route_data['Days_Real'].mean()
        std_lt = route_data['Days_Real'].std()
        
        print(f"\n📍 Simulating route: {high_risk_route[0]} - {high_risk_route[1]}")
        print(f"   Historical avg demand: {avg_demand:.1f} units")
        print(f"   Historical avg lead time: {avg_lt:.1f} days")
        
        # Run simulation
        np.random.seed(42)
        stockout_events = []
        inventory_costs = []
        
        safety_stock = 50  # Initial guess
        
        for _ in range(n_simulations):
            # Simulate demand and lead time
            demand = np.random.normal(avg_demand, std_demand)
            lead_time = np.random.normal(avg_lt, std_lt)
            
            # Total demand during lead time
            lead_time_demand = demand * (lead_time / 7)
            
            # Check if stockout occurs
            if lead_time_demand > safety_stock:
                stockout_events.append(1)
            else:
                stockout_events.append(0)
            
            # Calculate holding cost
            inventory_costs.append(safety_stock * 0.50 * 30)
        
        stockout_probability = np.mean(stockout_events)
        avg_inventory_cost = np.mean(inventory_costs)
        
        # Value at Risk (VaR) - 95th percentile
        var_95 = np.percentile(inventory_costs, 95)
        
        print(f"\n📊 Simulation Results:")
        print(f"   Stockout probability: {stockout_probability*100:.2f}%")
        print(f"   Average monthly holding cost: ${avg_inventory_cost:,.2f}")
        print(f"   VaR (95%): ${var_95:,.2f}")
        print(f"   Recommended safety stock: {safety_stock} units")
        
        self.simulation_results = {
            'stockout_prob': stockout_probability,
            'avg_cost': avg_inventory_cost,
            'var_95': var_95,
            'stockout_events': stockout_events,
            'inventory_costs': inventory_costs
        }
        
        return self.simulation_results
    
    def scenario_analysis(self):
        """What-if analysis for different disruption scenarios"""
        print("\n" + "=" * 80)
        print("SCENARIO ANALYSIS - DISRUPTION IMPACT MODELING")
        print("=" * 80)
        
        scenarios = {
            'Baseline': {'delay_increase': 0, 'cost_increase': 0},
            'Mild_Disruption': {'delay_increase': 0.15, 'cost_increase': 0.10},
            'Moderate_Disruption': {'delay_increase': 0.30, 'cost_increase': 0.25},
            'Severe_Disruption': {'delay_increase': 0.50, 'cost_increase': 0.50},
            'Black_Swan': {'delay_increase': 1.00, 'cost_increase': 1.00}
        }
        
        results = []
        
        for scenario_name, params in scenarios.items():
            # Simulate scenario impact
            simulated_delays = self.data['Delay_Days'].copy()
            simulated_costs = self.data['Transport_Cost'].copy()
            
            # Apply scenario multipliers
            simulated_delays = simulated_delays * (1 + params['delay_increase'])
            simulated_costs = simulated_costs * (1 + params['cost_increase'])
            
            # Calculate metrics
            avg_delay = simulated_delays.mean()
            total_cost = simulated_costs.sum()
            delay_rate = ((self.data['Days_Scheduled'] + simulated_delays) > self.data['Days_Scheduled']).mean()
            
            # Revenue impact (assuming 5% revenue loss per day of delay)
            revenue_impact = self.data['Sales_Value'].sum() * (avg_delay / 10) * 0.05
            
            results.append({
                'Scenario': scenario_name,
                'Avg_Delay_Days': avg_delay,
                'Total_Transport_Cost': total_cost,
                'Delay_Rate': delay_rate * 100,
                'Revenue_Impact': revenue_impact,
                'Total_Impact': total_cost + revenue_impact
            })
        
        scenario_df = pd.DataFrame(results)
        
        print("\n" + "-" * 80)
        print("SCENARIO COMPARISON")
        print("-" * 80)
        print(scenario_df.to_string(index=False))
        
        # Risk-adjusted recommendation
        print("\n💡 STRATEGIC RECOMMENDATIONS:")
        moderate_impact = scenario_df[scenario_df['Scenario'] == 'Moderate_Disruption']['Total_Impact'].values[0]
        baseline_impact = scenario_df[scenario_df['Scenario'] == 'Baseline']['Total_Impact'].values[0]
        
        risk_premium = moderate_impact - baseline_impact
        
        print(f"\n   Risk Premium (Moderate Disruption): ${risk_premium:,.2f}")
        print(f"   Recommended contingency budget: ${risk_premium * 0.3:,.2f} (30% of risk premium)")
        print(f"\n   Priority Actions:")
        print(f"   1. Build safety stock for routes with >40% delay probability")
        print(f"   2. Diversify suppliers in high-risk regions")
        print(f"   3. Establish alternate shipping modes for critical shipments")
        
        return scenario_df
    
    def supplier_diversification_analysis(self):
        """Recommend supplier diversification to reduce concentration risk"""
        print("\n" + "=" * 80)
        print("SUPPLIER DIVERSIFICATION STRATEGY")
        print("=" * 80)
        
        # Calculate supplier concentration
        supplier_analysis = self.data.groupby('Supplier_ID').agg({
            'Sales_Value': 'sum',
            'Order_ID': 'count',
            'Delivery_Status': 'mean',
            'Delay_Days': 'mean'
        }).reset_index()
        
        supplier_analysis.columns = ['Supplier_ID', 'Total_Revenue', 'Order_Count', 'Delay_Rate', 'Avg_Delay']
        supplier_analysis['Revenue_Share'] = (supplier_analysis['Total_Revenue'] / supplier_analysis['Total_Revenue'].sum()) * 100
        
        # Herfindahl-Hirschman Index (HHI) for concentration
        hhi = (supplier_analysis['Revenue_Share'] ** 2).sum()
        
        print(f"\n📊 Supplier Concentration Metrics:")
        print(f"   Total suppliers: {len(supplier_analysis)}")
        print(f"   HHI Score: {hhi:.2f} (>2500 = highly concentrated)")
        
        if hhi > 2500:
            concentration_level = "⚠️  HIGH - Significant concentration risk"
        elif hhi > 1500:
            concentration_level = "⚡ MODERATE - Some concentration risk"
        else:
            concentration_level = "✅ LOW - Well diversified"
        
        print(f"   Concentration Level: {concentration_level}")
        
        # Top 5 suppliers
        top_suppliers = supplier_analysis.nlargest(5, 'Revenue_Share')
        print("\n" + "-" * 80)
        print("TOP 5 SUPPLIERS (by Revenue)")
        print("-" * 80)
        print(top_suppliers[['Supplier_ID', 'Revenue_Share', 'Delay_Rate', 'Avg_Delay']].to_string(index=False))
        
        # Identify high-risk, high-dependency suppliers
        top_suppliers['Risk_Score'] = top_suppliers['Revenue_Share'] * top_suppliers['Delay_Rate'] * 100
        high_risk_suppliers = top_suppliers[top_suppliers['Risk_Score'] > 50]
        
        if len(high_risk_suppliers) > 0:
            print(f"\n🚨 HIGH-RISK SUPPLIERS (high dependency + high delays):")
            for _, row in high_risk_suppliers.iterrows():
                print(f"   {row['Supplier_ID']}: {row['Revenue_Share']:.1f}% revenue, {row['Delay_Rate']*100:.1f}% delays")
            
            print(f"\n   Recommendation: Reduce dependency on these suppliers by 30-50%")
        
        return supplier_analysis
# ============================================================================
# ADVANCED VISUALIZATION & REPORTING
# ============================================================================

class AdvancedVisualization:
    """Comprehensive visualization suite"""

    @staticmethod
    def create_executive_dashboard(data, model, feature_importance, scenario_df):
        fig = plt.figure(figsize=(20, 12))
        gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)

        # 1. Weekly delay trend
        ax1 = fig.add_subplot(gs[0, 0:2])
        weekly_delays = data.groupby(
            data['Order_Date'].dt.to_period('W')
        )['Delivery_Status'].mean()
        ax1.plot(weekly_delays.index.astype(str), weekly_delays.values, linewidth=2)
        ax1.set_title('Weekly Delay Rate Trend', fontweight='bold')
        ax1.tick_params(axis='x', rotation=45)
        ax1.grid(alpha=0.3)

        # 2. Country risk
        ax2 = fig.add_subplot(gs[0, 2:4])
        geo_risk = (
            data.groupby('Destination_Country')['Delivery_Status']
            .mean()
            .sort_values(ascending=False)
            .head(10)
        )
        ax2.barh(geo_risk.index, geo_risk.values)
        ax2.set_title('Top 10 Countries by Delay Rate', fontweight='bold')

        # 3. Feature importance
        ax3 = fig.add_subplot(gs[1, 0:2])
        top_features = feature_importance.head(10)
        ax3.barh(top_features['Feature'], top_features['Importance'])
        ax3.set_title('Top Predictive Features', fontweight='bold')
        ax3.invert_yaxis()

        # 4. Scenario impact
        ax4 = fig.add_subplot(gs[1, 2:4])
        ax4.bar(
            scenario_df['Scenario'],
            scenario_df['Total_Impact'] / 1e6
        )
        ax4.set_title('Scenario Impact ($M)', fontweight='bold')
        ax4.tick_params(axis='x', rotation=45)

        # 5. Shipping mode performance
        ax5 = fig.add_subplot(gs[2, 0])
        mode_perf = data.groupby('Shipping_Mode')['Delivery_Status'].mean()
        ax5.bar(mode_perf.index, mode_perf.values * 100)
        ax5.set_title('Delay Rate by Shipping Mode')

        # 6. Cost vs benefit
        ax6 = fig.add_subplot(gs[2, 1])
        ax6.scatter(
            data['Transport_Cost'],
            data['Benefit_per_Order'],
            alpha=0.4
        )
        ax6.set_title('Cost vs Benefit')

        # 7. Delay distribution
        ax7 = fig.add_subplot(gs[2, 2])
        delays = data[data['Delivery_Status'] == 1]['Delay_Days']
        ax7.hist(delays, bins=30)
        ax7.set_title('Delay Distribution')

        # 8. Risk score distribution (SAFE)
        ax8 = fig.add_subplot(gs[2, 3])
        if model is not None:
            feature_cols = [
                'Destination_Country', 'Shipping_Mode', 'Product_Category', 'Supplier_ID',
                'Days_Scheduled', 'Order_Quantity', 'Sales_Value', 'Transport_Cost',
                'Route_Volatility', 'Country_Risk_Score', 'Mode_Risk_Score',
                'Supplier_Risk_Score', 'Month', 'Quarter', 'Day_of_Week',
                'Is_Peak_Season', 'Is_Winter', 'MA_30_Delay_Rate', 'MA_90_Delay_Rate',
                'Value_Quantity_Ratio', 'Cost_Benefit_Ratio', 'Priority_Risk', 'EMA_Delay'
            ]

            if 'Supplier_Centrality' in data.columns:
                feature_cols.extend(['Supplier_Centrality', 'Warehouse_Centrality'])

            X_full = data[feature_cols].copy()

            for col in [
                'Destination_Country',
                'Shipping_Mode',
                'Product_Category',
                'Supplier_ID'
            ]:
                le = LabelEncoder()
                X_full[col] = le.fit_transform(X_full[col].astype(str))

            X_full = X_full.fillna(X_full.median())
            X_full = X_full[model.feature_names_in_]

            risk_scores = model.predict_proba(X_full)[:, 1]
            ax8.hist(risk_scores, bins=50)
            ax8.axvline(0.5, linestyle='--', color='red')
            ax8.set_title('Predicted Risk Score Distribution')

        plt.suptitle(
            'SUPPLY CHAIN EXECUTIVE DASHBOARD',
            fontsize=18,
            fontweight='bold'
        )
        plt.savefig(
            'executive_dashboard.png',
            dpi=300,
            bbox_inches='tight'
        )
        print("✓ Executive dashboard saved: executive_dashboard.png")
        return fig

    @staticmethod
    def create_network_visualization(network):
        fig, ax = plt.subplots(figsize=(16, 12))
        pos = nx.spring_layout(network, seed=42)

        nx.draw_networkx_nodes(network, pos, node_size=400, alpha=0.8, ax=ax)
        nx.draw_networkx_edges(network, pos, alpha=0.4, ax=ax)
        nx.draw_networkx_labels(network, pos, font_size=7, ax=ax)

        ax.set_title('Global Supply Chain Network Topology', fontweight='bold')
        ax.axis('off')

        plt.tight_layout()
        plt.savefig('network_topology.png', dpi=300, bbox_inches='tight')
        print("✓ Network visualization saved: network_topology.png")
        return fig

    @staticmethod
    def create_monte_carlo_visualization(simulation_results):
        if simulation_results is None:
            print("⚠️ No Monte Carlo results to visualize.")
            return None

        fig, ax = plt.subplots(figsize=(12, 6))
        costs = simulation_results['inventory_costs']
        var_95 = simulation_results['var_95']
        stockout_prob = simulation_results['stockout_prob']

        ax.hist(costs, bins=50, alpha=0.75, edgecolor='black')
        ax.axvline(var_95, linestyle='--', linewidth=2,
                   label=f'VaR 95% = ${var_95:,.0f}')
        ax.set_title('Monte Carlo Simulation – Inventory Risk', fontweight='bold')
        ax.set_xlabel('Monthly Holding Cost ($)')
        ax.set_ylabel('Frequency')

        ax.text(
            0.95, 0.95,
            f"Stockout Probability: {stockout_prob*100:.2f}%",
            transform=ax.transAxes,
            ha='right', va='top',
            bbox=dict(facecolor='white', alpha=0.8)
        )

        ax.legend()
        ax.grid(alpha=0.3)

        plt.tight_layout()
        plt.savefig('monte_carlo_simulation.png', dpi=300, bbox_inches='tight')
        print("✓ Monte Carlo visualization saved: monte_carlo_simulation.png")
        return fig



# ============================================================================
# MAIN EXECUTION PIPELINE - ADVANCED VERSION
# ============================================================================

def main_advanced():
    """Execute comprehensive advanced supply chain analysis"""
    
    print("\n")
    print("╔" + "=" * 78 + "╗")
    print("║" + " " * 78 + "║")
    print("║" + "  ADVANCED SUPPLY CHAIN RESILIENCE & DISRUPTION MODELING".center(78) + "║")
    print("║" + "  Enterprise-Grade Predictive & Prescriptive Analytics".center(78) + "║")
    print("║" + " " * 78 + "║")
    print("║" + "  New Features:".center(78) + "║")
    print("║" + "  • Multi-objective optimization".center(78) + "║")
    print("║" + "  • Monte Carlo risk simulation".center(78) + "║")
    print("║" + "  • Network topology analysis".center(78) + "║")
    print("║" + "  • Time series forecasting".center(78) + "║")
    print("║" + "  • Scenario analysis".center(78) + "║")
    print("║" + "  • Supplier diversification strategy".center(78) + "║")
    print("║" + " " * 78 + "║")
    print("╚" + "=" * 78 + "╝")
    print("\n")
    
    # ========== PHASE 1: ADVANCED DATA ENGINEERING ==========
    print("\n🔧 PHASE 1: Advanced Data Engineering & Network Analysis")
    print("-" * 80)
    
    kaggle_path = '/kaggle/input/us-data/data (1).csv' 
    engineer = AdvancedSupplyChainDataEngineer()
    data = engineer.load_and_map_data(kaggle_path)
    network = engineer.build_supply_chain_network()
    data = engineer.engineer_advanced_features()
    anomalies = engineer.detect_anomalies()
    
    # ========== PHASE 2: ADVANCED PREDICTIVE MODELING ==========
    print("\n🤖 PHASE 2: Advanced Predictive Modeling & Forecasting")
    print("-" * 80)
    
    predictor = AdvancedPredictiveModeling(data)
    predictor.prepare_modeling_data()
    predictor.train_with_hyperparameter_tuning()
    y_pred, y_pred_proba, feature_importance = predictor.advanced_model_evaluation()
    forecast = predictor.time_series_forecasting()
    
    # ========== PHASE 3: ADVANCED OPTIMIZATION ==========
    print("\n💡 PHASE 3: Multi-Objective Optimization & Risk Simulation")
    print("-" * 80)
    
    optimizer = AdvancedInventoryOptimization(data, predictor.model)
    optimized_routes = optimizer.multi_objective_optimization()
    simulation_results = optimizer.monte_carlo_risk_simulation(n_simulations=10000)
    scenario_df = optimizer.scenario_analysis()
    supplier_analysis = optimizer.supplier_diversification_analysis()
    
    # ========== PHASE 4: ADVANCED VISUALIZATION ==========
    print("\n📊 PHASE 4: Advanced Visualization & Reporting")
    print("-" * 80)
    
    viz = AdvancedVisualization()
    viz.create_executive_dashboard(data, predictor.model, feature_importance, scenario_df)
    viz.create_network_visualization(network)
    viz.create_monte_carlo_visualization(simulation_results)
    
    # ========== FINAL SUMMARY ==========
    print("\n")
    print("╔" + "=" * 78 + "╗")
    print("║" + " " * 78 + "║")
    print("║" + "  ADVANCED ANALYSIS COMPLETE".center(78) + "║")
    print("║" + " " * 78 + "║")
    print("║" + f"  Records Analyzed: {len(data):,}".center(78) + "║")
    print("║" + f"  Time Period: {data['Order_Date'].min().date()} to {data['Order_Date'].max().date()}".center(78) + "║")
    print("║" + f"  Model Performance (ROC-AUC): {roc_auc_score(predictor.y_test, y_pred_proba):.4f}".center(78) + "║")
    print("║" + f"  Network Nodes: {network.number_of_nodes()}  |  Edges: {network.number_of_edges()}".center(78) + "║")
    print("║" + f"  Anomalies Detected: {len(anomalies)} ({len(anomalies)/len(data)*100:.2f}%)".center(78) + "║")
    print("║" + " " * 78 + "║")
    print("║" + "  Generated Outputs:".center(78) + "║")
    print("║" + "    • executive_dashboard.png (8-panel comprehensive view)".center(78) + "║")
    print("║" + "    • network_topology.png (supply chain graph)".center(78) + "║")
    print("║" + "    • monte_carlo_simulation.png (risk quantification)".center(78) + "║")
    print("║" + " " * 78 + "║")
    print("║" + "  CSV Exports:".center(78) + "║")
    print("║" + "    • supply_chain_advanced_data.csv".center(78) + "║")
    print("║" + "    • optimized_routes.csv".center(78) + "║")
    print("║" + "    • scenario_analysis.csv".center(78) + "║")
    print("║" + "    • supplier_analysis.csv".center(78) + "║")
    print("║" + "    • feature_importance.csv".center(78) + "║")
    print("║" + " " * 78 + "║")
    print("╚" + "=" * 78 + "╝")
    print("\n")
    
    # Export results
    print("📁 Exporting results to CSV files...")
    data.to_csv('supply_chain_advanced_data.csv', index=False)
    optimized_routes.to_csv('optimized_routes.csv', index=False)
    scenario_df.to_csv('scenario_analysis.csv', index=False)
    supplier_analysis.to_csv('supplier_analysis.csv', index=False)
    feature_importance.to_csv('feature_importance.csv', index=False)
    print("✓ All results exported successfully")
    
    print("\n" + "=" * 80)
    print("✅ ALL PHASES COMPLETED SUCCESSFULLY!")
    print("=" * 80)
    print("\nThis analysis is ready for:")
    print("  • C-suite presentations")
    print("  • Academic portfolio submissions")
    print("  • Consulting case interviews")
    print("  • Technical deep-dives with stakeholders")
    print("\n" + "=" * 80)
    
    return {
        'data': data,
        'network': network,
        'model': predictor.model,
        'feature_importance': feature_importance,
        'optimized_routes': optimized_routes,
        'simulation_results': simulation_results,
        'scenario_analysis': scenario_df,
        'supplier_analysis': supplier_analysis,
        'anomalies': anomalies
    }


if __name__ == "__main__":
    results = main_advanced()
    import io
    from contextlib import redirect_stdout

    output_buffer = io.StringIO()

    with redirect_stdout(output_buffer):
        results = main_advanced()

    full_text_report = output_buffer.getvalue()

    print("Captured characters:", len(full_text_report))





    
    print("\n🎯 Next steps:")
    print("  1. Review executive_dashboard.png for key insights")
    print("  2. Examine CSV exports for detailed data")
    print("  3. Use scenario_analysis.csv for strategic planning")
    print("  4. Share network_topology.png to visualize dependencies")
    print("  5. Present monte_carlo_simulation.png for risk discussions")
    print("\n✨ Analysis complete - ready for stakeholder presentation!")

      



In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import textwrap

def render_text_page(lines, start, lines_per_page=55):
    fig = plt.figure(figsize=(8.27, 11.69))  # A4
    plt.axis("off")

    page_lines = lines[start:start + lines_per_page]
    wrapped = []
    for line in page_lines:
        wrapped.extend(textwrap.wrap(line, width=110) or [" "])

    plt.text(
        0.01, 0.99,
        "\n".join(wrapped),
        va="top",
        ha="left",
        family="monospace",
        fontsize=9
    )
    return fig


pdf_path = "/kaggle/working/supply_chain_full_execution_report.pdf"

with PdfPages(pdf_path) as pdf:

    # TEXT PAGES
    lines = full_text_report.split("\n")
    for i in range(0, len(lines), 55):
        fig = render_text_page(lines, i)
        pdf.savefig(fig)
        plt.close(fig)

    # FIGURE PAGES
    viz = AdvancedVisualization()

    fig1 = viz.create_executive_dashboard(
        results['data'],
        results['model'],
        results['feature_importance'],
        results['scenario_analysis']
    )
    pdf.savefig(fig1)
    plt.close(fig1)

    fig2 = viz.create_network_visualization(results['network'])
    pdf.savefig(fig2)
    plt.close(fig2)

    fig3 = viz.create_monte_carlo_visualization(results['simulation_results'])
    if fig3:
        pdf.savefig(fig3)
        plt.close(fig3)

print("✓ PDF saved at:", pdf_path)
